# A new notebook for day 12.
Goal: To implement A* properly instead of just implementing breatdh-first and being smart with what to check

In [ ]:
# Import packages
import numpy as np
curDay = 12

# Download input file
from download_inputs import download_inputs

# download_inputs(curDay)

In [ ]:
# Load input into workspace
filename = f'inputs\\day{curDay}.txt'
# curIn = np.loadtxt(filename)

inputFile = open(filename)
inputs = []
inputs = [s for s in inputFile.read().splitlines()]

print('Done loading input')

In [ ]:
# Load test-input into workspace
filenameTest = f'inputs\\day{curDay}test.txt'

inputFileTest = open(filenameTest)
inputsTest = []
inputsTest = [s for s in inputFileTest.read().splitlines()]

print('Done loading test-input') 

In [ ]:
curInput = inputs
# curInput = inputsTest

In [ ]:
# Start by finding S and E in inputs
wi = len(curInput[0])
he = len(curInput)
for x in range(wi):
    for y in range(he):
        curVal = curInput[y][x]

        if (curVal == 'S'):
            startPos = np.array([x,y])
        if (curVal == 'E'):
            EndPos = np.array([x,y])

In [ ]:
def getHeight(x,y):
    thisHeight = curInput[y][x]
    # print(thisHeight)

    if (thisHeight == 'S'):
        return ord('a')-1
    if (thisHeight == 'E'):
        return ord('z')
    else:
        return ord(thisHeight)

def getNeighbors(x,y):
    allNei = list()
    if x > 0:
        allNei.append([x-1,y])
    if x < wi-1:
        allNei.append([x+1,y])
    if y > 0:
        allNei.append([x,y-1])
    if y < he-1:
        allNei.append([x,y+1])
    allNei = np.stack(allNei,axis=0)
    return allNei

def getAllowedNei(x,y):
    thisHeight = getHeight(x,y)
    # print(thisHeight)
    curNei = getNeighbors(x,y)
    allNei = list()
    for nei in curNei:
        neiHei = getHeight(nei[0],nei[1])
        # print(neiHei)
        if neiHei <= thisHeight + 1:
            allNei.append(nei)
    return np.stack(allNei,axis=0)


In [ ]:
def getCameFromListBreadthFirst(startPos):
    curFront = list()
    curFront.append(startPos)
    curFront

    haveReached = list()
    haveReached.append(startPos)

    cameFrom = dict() 
    # cameFrom[startPos] = None
    cameFrom[startPos[0],startPos[1]] = None

    # print(haveReached) 
    # 
    # for k in range(10):
    while (len(curFront) > 0):

        curPos = curFront.pop(0)
        
        # # print(curPos)
        if curInput[curPos[1]][curPos[0]] == 'E':
        # if curInput[curPos[1]][curPos[0]] == 'E':
            # print('asdf')
            break

        curHeight = getHeight(curPos[0],curPos[1])
        # print(curPos)
        curNei = getNeighbors(curPos[0],curPos[1])
        # print(curNei)
        
        for nei in curNei:
            foundInFront = False
            
            if ((haveReached == nei).all(axis=1).any()):
                foundInFront = True

            neiHeight= getHeight(nei[0],nei[1])
            # print(neiHeight)
            # print(foundInFront)
            # print(curHeight)
            if (foundInFront == False) :
                # if np.abs(neiHeight - curHeight) <= 1:
                
                if neiHeight <= (curHeight +1):
                    curFront.append(nei)
                    haveReached.append(nei)

                    cameFrom[nei[0],nei[1]] = curPos
    return cameFrom

# Test
newCameFrom = getCameFromListBreadthFirst(startPos)


In [ ]:
def calcDistance(cameFrom):

    
    endFound = False
    for k in cameFrom.keys():
        # print(k == (EndPos[0],EndPos[1]))
        if (k == (EndPos[0],EndPos[1])):
            endFound = True 
    if endFound == False:
        return 1000 
    else: 
        numSteps = 0
        curStep = EndPos.copy()
        for k in range(1000):
            prevStep = cameFrom[(curStep[0],curStep[1])]
            # print(prevStep)
            curStep = prevStep
            numSteps += 1
            # print(len(curStep))
            if curStep is None:
                break 
            
        return numSteps -1 

calcDistance(newCameFrom)

In [ ]:
def getCameFromListDij(startPos):
    curFront = list()
    curFront.append(startPos)

    haveReached = list()
    haveReached.append(startPos)

    cameFrom = dict() 
    cameFrom[startPos[0],startPos[1]] = None

    # Make a list for priorities 
    prioList = list()
    prioList.append(0)

    # And a dict of costs
    costDict = dict()
    costDict[startPos[0],startPos[1]] = 0

 
    # for k in range(10): 
    while (len(curFront) > 0):

        # # Get the first element in the frontier
        # curPos = curFront.pop(0)
        # Instead, get the element with the lowest priority
        curMinimumPrio = np.argmin(prioList)
        prioList.pop(curMinimumPrio) # Pop element, but don't save
        curPos = curFront.pop(curMinimumPrio)
        
        if curInput[curPos[1]][curPos[0]] == 'E':
            break

        curHeight = getHeight(curPos[0],curPos[1])
        # print(curPos)
        curNei = getNeighbors(curPos[0],curPos[1])
        # print(curNei)

        # Get the current cost
        curCost = costDict[curPos[0],curPos[1]]
        
        for nei in curNei:

            # Get the next cost (Just add one, since no penalty)
            newCost = curCost + 1

            # Initialize flag
            foundInReached = False
            
            # Can keep the same method for checking, since no penalty for direction
            if ((haveReached == nei).all(axis=1).any()):
                foundInReached = True 

            if (foundInReached == False):
                    
                # Get the height
                neiHeight= getHeight(nei[0],nei[1])

                # if np.abs(neiHeight - curHeight) <= 1:
                if neiHeight <= (curHeight +1):
                    # Add to frontier
                    curFront.append(nei)
                    # Add to list of places checked
                    haveReached.append(nei)
                    # Add to cost dict with current cost
                    costDict[nei[0],nei[1]] = newCost 
                    # And add priority to priority list
                    prioList.append(newCost)

                    # Add to dict for direction
                    cameFrom[nei[0],nei[1]] = curPos
    return cameFrom

# Test
newCameFrom = getCameFromListDij(startPos)



In [ ]:
def calcHeu(curPos,endPos):
    # Simply Manhattan distance
    return np.abs(curPos[0]-endPos[0]) + np.abs(curPos[1]-endPos[1])
calcHeu(startPos,EndPos)

In [ ]:
# def getCameFromListAStar(startPos):
#     curFront = list()
#     curFront.append(startPos)

#     haveReached = list()
#     haveReached.append(startPos)

#     cameFrom = dict() 
#     cameFrom[startPos[0],startPos[1]] = None

#     # Make a list for priorities 
#     prioList = list()
#     prioList.append(0)

#     # # And a dict of costs
#     # costDict = dict()
#     # costDict[startPos[0],startPos[1]] = 0

#     curCost = 0
#     # for k in range(10): 
#     while (len(curFront) > 0):

#         # # Get the first element in the frontier
#         # curPos = curFront.pop(0)
#         # Instead, get the element with the lowest priority
#         curMinimumPrio = np.argmin(prioList)
#         prioList.pop(curMinimumPrio) # Pop element, but don't save
#         curPos = curFront.pop(curMinimumPrio)
        
#         if curInput[curPos[1]][curPos[0]] == 'E':
#             break

#         curHeight = getHeight(curPos[0],curPos[1])
#         # print(curPos)
#         curNei = getNeighbors(curPos[0],curPos[1])
#         # print(curNei)

#         # # Get the current cost
#         # curCost = costDict[curPos[0],curPos[1]]
        
#         for nei in curNei:

#             # Get the next cost (Just add one, since no penalty)
#             newCost = curCost + 1
#             # newCost = curCost + calcHeu(nei,EndPos) # A*
#             # newCost = calcHeu(nei,EndPos) # Greedy Best-First search

#             # Initialize flag
#             foundInReached = False
            
#             # Can keep the same method for checking if it's been cheched already
#             if ((haveReached == nei).all(axis=1).any()):
#                 foundInReached = True 
#             # # Also skip if cost is higher 
#             # if (newCost < curCost):
#             #     foundInReached = True


#             if (foundInReached == False):
                    
#                 # Get the height
#                 neiHeight= getHeight(nei[0],nei[1])

#                 # if np.abs(neiHeight - curHeight) <= 1:
#                 if neiHeight <= (curHeight +1):
#                     # Add to frontier
#                     curFront.append(nei)
#                     # Add to list of places checked
#                     haveReached.append(nei)
#                     # # Add to cost dict with current cost
#                     # costDict[nei[0],nei[1]] = newCost 
#                     # # And add priority to priority list
#                     # prioList.append(newCost)
#                     # And add priority to priority list
#                     prioList.append(newCost + calcHeu(nei,EndPos))

#                     # Add to dict for direction
#                     cameFrom[nei[0],nei[1]] = curPos
#         curCost = newCost 
#     return cameFrom

# # Test
# newCameFrom = getCameFromListAStar(startPos)



In [ ]:
# def getCameFromListAStar(startPos):
#     curFront = list()
#     curFront.append(startPos)

#     haveReached = list()
#     haveReached.append(startPos)

#     cameFrom = dict() 
#     cameFrom[startPos[0],startPos[1]] = None

#     # Make a list for priorities 
#     prioList = list()
#     prioList.append(0)

#     # And a dict of costs
#     costDict = dict()
#     costDict[startPos[0],startPos[1]] = 0

#     curCost = 0
#     # for k in range(10): 
#     while (len(curFront) > 0):

#         # # Get the first element in the frontier
#         # curPos = curFront.pop(0)
#         # Instead, get the element with the lowest priority
#         curMinimumPrio = np.argmin(prioList)
#         prioList.pop(curMinimumPrio) # Pop element, but don't save
#         curPos = curFront.pop(curMinimumPrio)
        
#         if curInput[curPos[1]][curPos[0]] == 'E':
#             break

#         curHeight = getHeight(curPos[0],curPos[1])
#         # print(curPos)
#         curNei = getNeighbors(curPos[0],curPos[1])
#         # print(curNei)

#         # Get the current cost
#         curCost = costDict[curPos[0],curPos[1]]
        
#         for nei in curNei:

#             # Get the next cost (Just add one, since no penalty)
#             newCost = curCost + 1
#             # newCost = curCost + calcHeu(nei,EndPos) # A*
#             # newCost = calcHeu(nei,EndPos) # Greedy Best-First search

#             # Initialize flag
#             foundInReached = False
            
#             # Can keep the same method for checking if it's been cheched already
#             if ((haveReached == nei).all(axis=1).any()):
#                 foundInReached = True 
#             # # Also skip if cost is higher 
#             # if (newCost < curCost):
#             #     foundInReached = True


#             if (foundInReached == False):
                    
#                 # Get the height
#                 neiHeight= getHeight(nei[0],nei[1])

#                 # if np.abs(neiHeight - curHeight) <= 1:
#                 if neiHeight <= (curHeight +1):
#                     # Add to frontier
#                     curFront.append(nei)
#                     # Add to list of places checked
#                     haveReached.append(nei)
#                     # Add to cost dict with current cost
#                     costDict[nei[0],nei[1]] = newCost 
#                     # # And add priority to priority list
#                     # prioList.append(newCost)
#                     # And add priority to priority list
#                     prioList.append(newCost + calcHeu(nei,EndPos))

#                     # Add to dict for direction
#                     cameFrom[nei[0],nei[1]] = curPos
#     return cameFrom

# # Test
# newCameFrom = getCameFromListAStar(startPos)



In [ ]:
sP = (startPos[0],startPos[1])
eP = (EndPos[0],EndPos[1])

In [ ]:
def getCameFromListAStar(sP):
    curFront = list()
    curFront.append(sP)

    cameFrom = dict() 
    cameFrom[sP] = None

    # Make a list for priorities 
    prioList = list()
    prioList.append(0)

    # And a dict of costs
    costDict = dict()
    costDict[sP] = 0

    curCost = 0
    # for k in range(10): 
    while (len(curFront) > 0):

        # # Get the first element in the frontier
        # curPos = curFront.pop(0)
        # Instead, get the element with the lowest priority
        curMinimumPrio = np.argmin(prioList)
        prioList.pop(curMinimumPrio) # Pop element, but don't save
        curPos = curFront.pop(curMinimumPrio)
        
        if curInput[curPos[1]][curPos[0]] == 'E':
            break

        curHeight = getHeight(curPos[0],curPos[1])
        # print(curPos)
        curNei = getNeighbors(curPos[0],curPos[1])
        # print(curNei)

        # Get the current cost
        curCost = costDict[curPos]
        
        for nei in curNei:
            neiP = (nei[0],nei[1])

            # Get the next cost (Just add one, since no penalty)
            newCost = curCost + 1
            # newCost = curCost + calcHeu(nei,EndPos) # A*
            # newCost = calcHeu(nei,EndPos) # Greedy Best-First search

            # Initialize flag
            foundInReached = False
            if neiP not in costDict.keys():
                foundInReached = True 
            
            # # Can keep the same method for checking if it's been cheched already
            # if ((haveReached == nei).all(axis=1).any()):
            #     foundInReached = True 
            # # # Also skip if cost is higher 
            # # if (newCost < curCost):
            # #     foundInReached = True


            if (foundInReached == False):
                    
                # Get the height
                neiHeight= getHeight(nei[0],nei[1])

                # if np.abs(neiHeight - curHeight) <= 1:
                if neiHeight <= (curHeight +1):
                    # Add to frontier
                    curFront.append(nei)
                    # # Add to list of places checked
                    # haveReached.append(nei)
                    # Add to cost dict with current cost
                    costDict[neiP] = newCost 
                    # # And add priority to priority list
                    # prioList.append(newCost)
                    # And add priority to priority list
                    prioList.append(newCost + calcHeu(nei,EndPos))

                    # Add to dict for direction
                    cameFrom[neiP] = curPos
    return cameFrom

# Test
newCameFrom = getCameFromListAStar(sP)


calcDistance(newCameFrom)

In [ ]:
newCameFrom

In [ ]:
qwer = (0,20)


# Old below

In [ ]:
asdfasdfasdf

In [ ]:
curFront = list()
curFront.append(startPos)
curFront

haveReached = list()
haveReached.append(startPos)

cameFrom = dict() 
# cameFrom[startPos] = None
cameFrom[startPos[0],startPos[1]] = None

# print(haveReached) 
# 
# for k in range(10):
while (len(curFront) > 0):

    curPos = curFront.pop(0)
    
    # # print(curPos)
    if curInput[curPos[1]][curPos[0]] == 'E':
    # if curInput[curPos[1]][curPos[0]] == 'E':
        print('asdf')
        break

    curHeight = getHeight(curPos[0],curPos[1])
    # print(curPos)
    curNei = getNeighbors(curPos[0],curPos[1])
    # print(curNei)
    
    for nei in curNei:
        foundInFront = False
        # print('-')
        # print(nei)
        # print('haveReached: ',haveReached)
        # print(haveReached == nei)
        # print((haveReached == nei).all(axis=1).any())
        if ((haveReached == nei).all(axis=1).any()):
            foundInFront = True

        neiHeight= getHeight(nei[0],nei[1])
        # print(neiHeight)
        # print(foundInFront)
        # print(curHeight)
        if (foundInFront == False) :
            # if np.abs(neiHeight - curHeight) <= 1:
            
            if neiHeight <= (curHeight +1):
                curFront.append(nei)
                haveReached.append(nei)

                cameFrom[nei[0],nei[1]] = curPos
            # if neiHeight > curHeight:
            #     curFront.append(nei)
            #     haveReached.append(nei)

            #     cameFrom[nei[0],nei[1]] = curPos

In [ ]:
cameFrom[(EndPos[0],EndPos[1])]

numSteps = 0
curStep = EndPos.copy()
for k in range(1000):
    prevStep = cameFrom[(curStep[0],curStep[1])]
    # print(prevStep)
    curStep = prevStep
    numSteps += 1
    # print(len(curStep))
    if curStep is None:
        break 
# numSteps -= 1
p1 = numSteps - 1
p1

In [ ]:
# cameFrom[(EndPos[0],EndPos[1])]
cameBackup = cameFrom.copy()

# Part 2

In [ ]:
def getCameFromList(startPos):
    curFront = list()
    curFront.append(startPos)
    curFront

    haveReached = list()
    haveReached.append(startPos)

    cameFrom = dict() 
    # cameFrom[startPos] = None
    cameFrom[startPos[0],startPos[1]] = None

    # print(haveReached) 
    # 
    # for k in range(10):
    while (len(curFront) > 0):

        curPos = curFront.pop(0)
        
        # # print(curPos)
        if curInput[curPos[1]][curPos[0]] == 'E':
        # if curInput[curPos[1]][curPos[0]] == 'E':
            # print('asdf')
            break

        curHeight = getHeight(curPos[0],curPos[1])
        # print(curPos)
        curNei = getNeighbors(curPos[0],curPos[1])
        # print(curNei)
        
        for nei in curNei:
            foundInFront = False
            
            if ((haveReached == nei).all(axis=1).any()):
                foundInFront = True

            neiHeight= getHeight(nei[0],nei[1])
            # print(neiHeight)
            # print(foundInFront)
            # print(curHeight)
            if (foundInFront == False) :
                # if np.abs(neiHeight - curHeight) <= 1:
                
                if neiHeight <= (curHeight +1):
                    curFront.append(nei)
                    haveReached.append(nei)

                    cameFrom[nei[0],nei[1]] = curPos
    return cameFrom
# newCameFrom = getCameFromList(startPos)

# newCameFrom =

In [ ]:

# testStart = np.array([150,20])
# testStart = np.array([153,0])
# startPos
# testStart
# testCameFrom = getCameFromList(testStart)

In [ ]:
def calcDistance(cameFrom):

    
    endFound = False
    for k in cameFrom.keys():
        # print(k == (EndPos[0],EndPos[1]))
        if (k == (EndPos[0],EndPos[1])):
            endFound = True 
    if endFound == False:
        return 1000 
    else: 
        numSteps = 0
        curStep = EndPos.copy()
        for k in range(1000):
            prevStep = cameFrom[(curStep[0],curStep[1])]
            # print(prevStep)
            curStep = prevStep
            numSteps += 1
            # print(len(curStep))
            if curStep is None:
                break 
            
        return numSteps -1 

# calcDistance(cameBackup)
# calcDistance(newCameFrom)
# calcDistance(testCameFrom)

In [ ]:
# curInput[0][153]
# # testCameFrom[EndPos[0],EndPos[1]]
# endFound = False
# for k in testCameFrom.keys():
#     print(k == (EndPos[0],EndPos[1]))
#     if (k == (EndPos[0],EndPos[1])):
#         endFound = True 
# if endFound == False:
#     return 1000


In [ ]:

curCameFrom = getCameFromList(np.array([0,1]))
# print(curCameFrom)
curDist = calcDistance(curCameFrom)

In [ ]:
curDist
startPos

In [ ]:
curMin = 1001
numA = 0
# for x in range(wi):
# for x in range(wi-1,0,-1):
for x in range(0,3,1):
    # Since we shouldn't start in the far left
    # if x > 70:
    # for y in range(he):
    for y in range(24,he):
        if curInput[y][x] == 'a':
            numA += 1
            # print(x,y)
            curCameFrom = getCameFromList(np.array([x,y]))
            # print(curCameFrom)
            curDist = calcDistance(curCameFrom)

            # print(curDist)
            if curDist < curMin:
                curMin = curDist
                print('Shortest distance so far:' ,curDist)
# numA 

# Old below

In [ ]:
# def getDistanceToEnd(startPos):
#     curFront = list()
#     curFront.append(startPos)
#     curFront

#     haveReached = list()
#     haveReached.append(startPos)

#     cameFrom = dict() 
#     # cameFrom[startPos] = None
#     cameFrom[startPos[0],startPos[1]] = None

#     # print(haveReached) 
#     # 
#     # for k in range(10):
#     while (len(curFront) > 0):

#         curPos = curFront.pop(0)
        
#         # # print(curPos)
#         if curInput[curPos[1]][curPos[0]] == 'E':
#         # if curInput[curPos[1]][curPos[0]] == 'E':
#             print('asdf')
#             break

#         curHeight = getHeight(curPos[0],curPos[1])
#         # print(curPos)
#         curNei = getNeighbors(curPos[0],curPos[1])
#         # print(curNei)
        
#         for nei in curNei:
#             foundInFront = False
#             # print('-')
#             # print(nei)
#             # print('haveReached: ',haveReached)
#             # print(haveReached == nei)
#             # print((haveReached == nei).all(axis=1).any())
#             if ((haveReached == nei).all(axis=1).any()):
#                 foundInFront = True

#             neiHeight= getHeight(nei[0],nei[1])
#             # print(neiHeight)
#             # print(foundInFront)
#             # print(curHeight)
#             if (foundInFront == False) :
#                 # if np.abs(neiHeight - curHeight) <= 1:
                
#                 if neiHeight <= (curHeight +1):
#                     curFront.append(nei)
#                 haveReached.append(nei)

#                 cameFrom[nei[0],nei[1]] = curPos
#             # if neiHeight > curHeight:
#             #     curFront.append(nei)
#             #     haveReached.append(nei)

#             #     cameFrom[nei[0],nei[1]] = curPos

#     print(cameFrom)
#     numSteps = 0
#     curStep = EndPos.copy()
#     for k in range(1000):
#         prevStep = cameFrom[(curStep[0],curStep[1])]
#         # print(prevStep)
#         curStep = prevStep
#         numSteps += 1
#         # print(len(curStep))
#         if curStep is None:
#             break 
#     # numSteps -= 1
#     # p1 = numSteps - 1
#     return numSteps-1

# # getDistanceToEnd(np.array([153,0]))
# getDistanceToEnd(startPos)

In [ ]:
# # curInput
# # curInput[startP]
# cameFrom[(EndPos[0],EndPos[1])]
# cameFrom[(EndPos[0],20)]
# # wi

In [ ]:
# print(EndPos)
# print(cameFrom[(EndPos[0],EndPos[1])])

# # curInput[20][131]
# # haveReached[-5:]
# # print(getHeight(131,20))
# print(getHeight(4,2))
# print(getHeight(EndPos[0],EndPos[1]))

In [ ]:
# # print(EndPos)
# print(cameFrom[4,2])

# # curInput[20][131]
# # haveReached[-5:]
# # print(getHeight(131,20))
# print(getHeight(4,2))
# print(getHeight(4,1))
# # print(getHeight(EndPos[0],EndPos[1]))

In [ ]:
# # # cameFrom[(EndPos[0],EndPos[1])]
# # curHeight = ord('x')
# # neiHeight = ord('z')
# # # neiHeight = getHeight(EndPos[0],EndPos[1])

# if neiHeight <= (curHeight +1):
#     print('asdf')
# # if neiHeight > curHeight:
# #     print('qwer')
# # ord('z')
# # ord('x')

In [ ]:

# # for x in range(wi):
# #     print('')
# #     print(cameFrom[x,0])
# #     if cameFrom[x,0] is not None:
# #         # print(cameFrom[x,y])
# #         if cameFrom[x,0][0] < x:
# #             print('>',end='')

# print('S',end='')
# for y in range(he):
#     for x in range(wi):
#         # print(cameFrom[x,y])
#         if cameFrom[x,y] is not None:
#             # print(cameFrom[x,y])
#             if cameFrom[x,y][0] < x:
#                 print('<',end='')
#             if cameFrom[x,y][0] > x:
#                 print('>',end='')
#             if cameFrom[x,y][1] < y:
#                 print('^',end='')
#             if cameFrom[x,y][1] > y:
#                 print('v',end='')
                
#     print('')
# # for y in range(he):
# #     print('')
# #     print(cameFrom[0,y])
# #     # for x in range(wi):
# #     #     # print(cameFrom[x,y])
# #     #     if cameFrom[x,y] is not None:
# #     #         # print(cameFrom[x,y])
# #     #         if cameFrom[x,y][0] < x:
# #     #             print('<',end='')


In [ ]:
# # curPos = [0,0]
# x = 7
# y = 0
# cameFrom[x,y]


In [ ]:
# cameFrom[(EndPos[0],EndPos[1])]

# numSteps = 0
# curStep = EndPos.copy()
# for k in range(100):
#     prevStep = cameFrom[(curStep[0],curStep[1])]
#     # print(prevStep)
#     curStep = prevStep
#     numSteps += 1
#     # print(len(curStep))
#     if curStep is None:
#         break 
# # numSteps -= 1
# p1 = numSteps - 1
# p1

In [ ]:

                # cameFrom[curPos[0],curPos[1]] = 
                # print('asdf')
    
            # if (getHeight(nei[0],nei[1]) == ord('E')):
            #     print(curPos)
            #     print('done')
            #     break
    # print('--')
    # print('fr: ',curFront)
    # print('hr: ',haveReached)
    # print('--')

# print('fr: ',curFront)
# print('hr: ',haveReached)
# cameFrom

# curPos
# nei
# curInput[0][5]

In [ ]:
# cameFrom[(EndPos[0],EndPos[1])]
# curPos 
# for d in cameFrom:
#     print(d)
# EndPos

In [ ]:
# curInput[0][5]